<a href="https://colab.research.google.com/github/kirillel85/kirillel85/blob/public_main/API_HH_prof_roles.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
import requests as req
import matplotlib
import json
from tqdm import tqdm
import time

### Запрос полного справочника professional_roles (Дерево профессий и категорий профессиональных ролей)

In [3]:
response_curl = ! curl -H 'User-Agent: api-test-agent' 'https://api.hh.ru/professional_roles'

# используем метод spstr что бы получить строку из списка IPython.utils.text.SList
# Потом из этой строки получаем json методом json.loads()
response = json.loads(response_curl.spstr)
print(f"Тип данных response до фильтрации: {type(response)}")
print(f"Список ключей: {response.keys()}")

# Фильтруем по категории
response = response['categories']
print(f"Тип данных response после фильтрации ['categories']: {type(response)}")

Тип данных response до фильтрации: <class 'dict'>
Список ключей: dict_keys(['categories'])
Тип данных response после фильтрации ['categories']: <class 'list'>


In [4]:
# response_curl

In [ ]:
# response
# [{'id': '19',
#   'name': 'Автомобильный бизнес',
#   'roles': [{'id': '4',
#     'name': 'Автомойщик',
#     'accept_incomplete_resumes': True,
#     'is_default': False,
#     'select_deprecated': False,
#     'search_deprecated': False},
#    {'id': '5',
#     'name': 'Автослесарь, автомеханик',
#     'accept_incomplete_resumes': True,
#     'is_default': False,
#     'select_deprecated': False,
#     'search_deprecated': False},
#    {'id': '62',
#     'name': 'Мастер-приемщик',
#     'accept_incomplete_resumes': True,
#     'is_default': False,
#     'select_deprecated': False,
#     'search_deprecated': False},
#    {'id': '70',
#     'name': 'Менеджер по продажам, менеджер по работе с клиентами',
#     'accept_incomplete_resumes': False,
#     'is_default': False,
#     'select_deprecated': False,
#     'search_deprecated': False}]},
#  {'id': '5',
#   'name': 'Административный персонал',
#   'roles': [{'id': '8', .........................................

In [18]:
final_dataframe = pd.DataFrame() # Создаём пустой фрейм

for dicts_in_list in response: # Распаковываю список словарей и в следующем цикле итеррирую обьекты

  # print(dicts_in_list)
  # print('----------'*10)

  # Получаем такую колбасу в одном цикле итеррации
  # {'id': '19', 'name': 'Автомобильный бизнес', 'roles': [{'id': '4', 'name': 'Автомойщик', 'accept_incomplete_resumes': True, 'is_d...........

  for key, value in dicts_in_list.items():

    category_id = dicts_in_list['id']
    category_name = dicts_in_list['name']

    # print(key, value)
    # print('----------'*10)
    # Получаем в третьем по счету value такое:
    #  {'id': '19', 'name': 'Автомобильный бизнес', 'roles': [{'id': '4', 'name': 'Автомойщик', 'accept_incomplete_resumes': True, 'is_default': False, 'select_deprecated': False, 'search_deprecated': False}, {'id': '5', 'name': 'Автослесарь, автомеханик', 'accept_incomplete_resumes': True, 'is_default': False, 'select_deprecated': False, 'search_deprecated': False}, {'id': '62', 'name': 'Мастер-приемщик', 'accept_incomplete_resumes': True, 'is_default': False, 'select_deprecated': False, 'search_deprecated': False}, {'id': '70', 'name': 'Менеджер по продажам, менеджер по работе с клиентами', 'accept_incomplete_resumes': False, 'is_default': False, 'select_deprecated': False, 'search_deprecated': False}]}

    # Мы проходим циклом по вложенному списку словарей с названием рабочих специальностей
    if key == 'roles':  # Если ключ в словаре == 'roles', то иттерировать список в значениях ключа
      for dicts_in_list2 in dicts_in_list['roles']:
      # Получаем:  {'id': '4', 'name': 'Автомойщик', 'accept_incomplete_resumes': True, 'is_default': False, 'select_d.................
        profession_id = dicts_in_list2['id']
        profession_name = dicts_in_list2['name']

        cycle_df = pd.json_normalize(dicts_in_list2)[['id','name']] # Превращаем в датафрейм
        cycle_df.columns = [['profession_id', 'profession_name']]
        cycle_df.insert(0, 'category_id', category_id) # Добавляем поле 'category_id'
        cycle_df.insert(1, 'category_name', category_name) # Добавляем поле 'category_name'

        final_dataframe = pd.concat([final_dataframe, cycle_df], ignore_index=True) #  Соединяем все датафреймы в цикле
print('Датафрейм получен')
print(f'Длина строк {len(final_dataframe)}')


Датафрейм получен
Длина строк 270


In [ ]:
# cycle_df

In [ ]:
# final_dataframe.head()

#### Получили итоговый датафрейм

In [19]:
print(f'Строк всего: - {len(final_dataframe)}')
final_dataframe.to_excel('/content/final_dataframe.xlsx', sheet_name='Справочник HH', index=True)

print('Датафрейм выгружен в Excel')

Строк всего: - 270
Датафрейм выгружен в Excel


#### Запросы через curl / Linux и направление потока в текстовый файл

In [ ]:
# ! curl --help all >> curl_help1.txt
# # Записываем вывод команды в текстовый файл

In [ ]:
# ! cat /content/curl_help.txt | grep 'H'
# # Ищем в этом текстовом файле ключ H

In [ ]:
# !whatis wc

In [ ]:
# Тоесть в линуксе запрос выглядел бы так
# curl -H 'User-Agent: api-test-agent' 'https://api.hh.ru/professional_roles' >> curl_json.txt | (перенаправление на поток который открывает скрипт пайтон)
# Надо подумать как в скрипте пайтона можно обратится к утелите curl когда ты находишься в Linux

https://blog.parente.dev/jupyter-tidbit-ipython-slists/
обработка  в пайтоне потоков вывода системных команд линукс. (Для инфо)

### Запрос профессиональные роли в категории "Информационные технологии" по общему User-Agent

In [ ]:
# Запрашиваем справочник через cURL
response_curl2 = ! curl -H 'User-Agent: api-test-agent' 'https://api.hh.ru/professional_roles'
# удалил ключ -k   --insecure           Allow insecure server connections
# -H, --header <header/@file> Pass custom header(s) to server

In [ ]:
response_curl2 == response_curl2.l == response_curl2.list == response_curl2.get_list()
# способы из статьи выше

True

In [ ]:
# response_curl

In [ ]:
# type(response_curl)

In [ ]:
response2 = json.loads(response_curl2.spstr)
# используем метод spstr что бы получить строку из списка IPython.utils.text.SList
# Потом из этой строки получаем json методом json.loads()

In [ ]:
print(type(response2))
print(response2.keys())

<class 'dict'>
dict_keys(['categories'])


In [ ]:
# response2

In [ ]:
# Фильтруем по категории
response2 = response2['categories']
print(type(response2))

<class 'list'>


In [ ]:
pd.json_normalize(response2)
# метод pd для создания датафрейма

,id,name,roles
0,19,Автомобильный бизнес,"[{'id': '4', 'name': 'Автомойщик', 'accept_inc..."
1,5,Административный персонал,"[{'id': '8', 'name': 'Администратор', 'accept_..."
2,15,Безопасность,"[{'id': '22', 'name': 'Военнослужащий', 'accep..."
3,26,Высший и средний менеджмент,"[{'id': '26', 'name': 'Генеральный директор, и..."
4,8,Добыча сырья,"[{'id': '27', 'name': 'Геодезист', 'accept_inc..."
5,16,"Домашний, обслуживающий персонал","[{'id': '8', 'name': 'Администратор', 'accept_..."
6,14,Закупки,"[{'id': '66', 'name': 'Менеджер по закупкам', ..."
7,11,Информационные технологии,"[{'id': '156', 'name': 'BI-аналитик, аналитик ..."
8,24,"Искусство, развлечения, массмедиа","[{'id': '12', 'name': 'Арт-директор, креативны..."
9,6,"Маркетинг, реклама, PR","[{'id': '1', 'name': 'Event-менеджер', 'accept..."


In [ ]:
# pd.DataFrame(response[2])['roles']
# # Смотрим записи в json по определенному индексу в списке

In [ ]:
# print(*response, sep= '\n')
# # смотрим весь список словарей

In [ ]:
dataframes = pd.DataFrame()

for dicts_in_list in response: # Распаковываю список словарей и в следующем цикле итерирую обьекты
  # print(dicts_in_list)
  for key, value in dicts_in_list.items():
    category_name = value   # Имя категории в которой мы проходим
    # Получаем такое
    #  {'id': '19', 'name': 'Автомобильный бизнес', 'roles': [{'id': '4', 'name': 'Автомойщик', 'accept_incomplete_resumes': True, 'is_default': False, 'select_deprecated': False, 'search_deprecated': False}, {'id': '5', 'name': 'Автослесарь, автомеханик', 'accept_incomplete_resumes': True, 'is_default': False, 'select_deprecated': False, 'search_deprecated': False}, {'id': '62', 'name': 'Мастер-приемщик', 'accept_incomplete_resumes': True, 'is_default': False, 'select_deprecated': False, 'search_deprecated': False}, {'id': '70', 'name': 'Менеджер по продажам, менеджер по работе с клиентами', 'accept_incomplete_resumes': False, 'is_default': False, 'select_deprecated': False, 'search_deprecated': False}]}

    # print(f"{key}: {value}")
    # print(category_name)

    if "Информационные технологии" == value: # В этом цикле проходим по имени категории и выбираем только Информационные технологии
      # print(f"{key}: {value}")
      # print('----------'*10)

      # Мы проходим циклом по вложенному списку словарей с названием рабочих специальностей в категории Информационные технологии (Бизнес-аналитик, Аналитик и т.д.)
      for dicts_in_list2 in dicts_in_list['roles']:

        # print(dicts_in_list2)  # Печать словарей с названием рабочих специальностей в категории
        # К примеру:
        # {'id': '156', 'name': 'BI-аналитик, аналитик данных', 'accept_incomplete_resumes': False, 'is_default': False, 'select_deprecated': False, 'search_deprecated': False}
        # {'id': '160', 'name': 'DevOps-инженер', 'accept_incomplete_resumes': False, 'is_default': False, 'select_deprecated': False, 'search_deprecated': False}


        df = pd.json_normalize(dicts_in_list2)[['id','name']] # Превращаем в датафрейм
        df.insert(1, 'category_name', category_name) # Добавляем поле 'category_name'
        # print(type(df))

        dataframes = pd.concat([dataframes, df], ignore_index=True) #  Соединяем в все датавреймы в цикле
      # print('----------'*10)

In [ ]:
df

,id,category_name,name
0,126,Информационные технологии,Технический писатель


#### Итоговый датафрейм получен

In [ ]:
# dataframes

In [ ]:
print(f'Строк всего: - {len(dataframes)}')
dataframes.to_excel('/content/dataframes.xlsx', sheet_name='Справочник HH', index=True)

print('Датафрейм выгружен в Excel')

Строк всего: - 25
Датафрейм выгружен в Excel


##### Разбираюсь с итеррацией словарей в цикле

In [ ]:
# Метод pd.DataFrame() кушает только словари со значениями в виде списка
d = {'id': ['156'], 'name': ['BI-аналитик, аналитик данных'], 'accept_incomplete_resumes': False, 'is_default': False, 'select_deprecated': False, 'search_deprecated': False}
df1 = pd.DataFrame(d)
df1

,id,name,accept_incomplete_resumes,is_default,select_deprecated,search_deprecated
0,156,"BI-аналитик, аналитик данных",False,False,False,False


In [ ]:
# А вот в виде строки не кушает
# Ошибка:  If using all scalar values, you must pass an index
dd = {'id': '156', 'name': 'BI-аналитик, аналитик данных', 'accept_incomplete_resumes': False, 'is_default': False, 'select_deprecated': False, 'search_deprecated': False}
df2 = pd.DataFrame(dd)
df2

In [ ]:
print(type(list(dd.items())))
print('----------'*10)
list(dd.items())

<class 'list'>
----------------------------------------------------------------------------------------------------


[('id', '156'),
 ('name', 'BI-аналитик, аналитик данных'),
 ('accept_incomplete_resumes', False),
 ('is_default', False),
 ('select_deprecated', False),
 ('search_deprecated', False)]

In [ ]:
# В виде листа транспонирует
df3 = pd.DataFrame(list(dd.items()),  columns=[['id', 'name']])
df3
# , columns=[['id', 'name']]

,id,name
0,id,156
1,name,"BI-аналитик, аналитик данных"
2,accept_incomplete_resumes,False
3,is_default,False
4,select_deprecated,False
5,search_deprecated,False


In [ ]:
# # Метод pd.DataFrame.from_dict работает так же как и метод pd.DataFrame()
# # Ошибка:  If using all scalar values, you must pass an index
# d_dict = {'id': '156', 'name': 'BI-аналитик, аналитик данных', 'accept_incomplete_resumes': False, 'is_default': False, 'select_deprecated': False, 'search_deprecated': False}
# pd.DataFrame.from_dict((d_dict))

In [ ]:
data = {'row_1': [3, 2, 1, 0], 'row_2': ['a', 'b', 'c', 'd']}

pd.DataFrame.from_dict(data, orient='Columns')

# Вывод  orient='index'
#        A  B  C  D
# row_1  3  2  1  0
# row_2  a  b  c  d

# Вывод  orient='Columns'
# 	row_1	row_2
# 0 	3	  a
# 1	  2	  b
# 2	  1	  c
# 3	  0	  d

In [ ]:
# dict_data = {
#     '2022-01-01': 100,
#     '2022-01-02': 200,
#     '2022-01-03': 300,
#     '2022-01-04': 400,
#     '2022-01-05': 500
# }

# # преобразование словаря в DataFrame
# df = pd.DataFrame(list(dict_data.items()), columns=['Date', 'Value'])

# # вывод DataFrame
# print(df)

         Date  Value
0  2022-01-01    100
1  2022-01-02    200
2  2022-01-03    300
3  2022-01-04    400
4  2022-01-05    500
